In [62]:
import os
from pathlib import Path
# from matplotlib import pyplot as plt
# from matplotlib import cm
import pandas as pd
import yaml
# import glob
# import seaborn as sns
from Bio import SeqIO
from Bio.Seq import Seq

In [49]:
proj_dir="/master/nplatt/sch_hae_its-nigeria"
results_dir=f"{proj_dir}/results"

In [50]:
info_df=pd.read_csv(f"{proj_dir}/its-nigeria_samplesheet.csv", sep=",")
info_df

,wgs_id,its_id,sra,species,country,locale,patient,miracidum
0,Sb_NG_ak_1.1,Sb.ng.ak.1.1F,NaN,sbovis,nigeria,ak,1,1
1,Sb_NG_ak_2.1,sb_ng_ak_2.1,NaN,sbovis,nigeria,ak,2,1
2,Sb_NG_ak_2.2,Sb.ng.ak.2.2F,NaN,sbovis,nigeria,ak,2,2
3,Sb_NG_ak_2.3,Sb.ng.ak.2.3F,NaN,sbovis,nigeria,ak,2,3
4,Sb_NG_ak_3.1,Sb.ng.ak.3.1R,NaN,sbovis,nigeria,ak,3,1
...,...,...,...,...,...,...,...,...
200,Sh_NG_os_3_1,sh_ng_os_3_1,NaN,shaematobium,nigeria,osun,3,1
201,c_Sh_NG_os_3_11,Sh.ng.os.3.11F,NaN,shaematobium,nigeria,osun,3,11
202,c_Sh_NG_os_3_5,Sh.ng.os.3.5F,NaN,shaematobium,nigeria,osun,3,5
203,c_Sh_NG_os_3_6,Sh.ng.os.3.6F,NaN,shaematobium,nigeria,osun,3,6


In [ ]:
#choose samples 

In [54]:
phylo_df=pd.read_csv("../svdq/phylo.samples.list", sep=",", header=None)
samples = list(phylo_df[0])
samples[:10]

['Sb_NG_ak_1.1',
 'Sb_NG_ak_2.1',
 'Sb_NG_ak_2.2',
 'Sb_NG_ak_2.3',
 'Sb_NG_ak_3.1',
 'Sb_NG_au_1.2',
 'Sb_NG_au_1.1',
 'Sb_NG_au_2.2',
 'Sb_NG_au_2.11',
 'Sb_NG_au_2.12']

In [58]:
len(samples)

178

# Whole Mitochondria phylogeny

In [113]:
Path(f"{results_dir}/mito_phylogeny").mkdir(parents=True, exist_ok=True)
os.chdir(f"{results_dir}/mito_phylogeny")

In [93]:
%%bash

conda run -n mito_assembly --live-stream clustalo \
    -INFILE=../mito_assembly/mito_assemblies.fasta \
    -OUTFILE=mito.aligned.fasta \
    -OUTPUT=FASTA \
    --threads=12 

Using 12 threads
Read 205 sequences (type: DNA) from ../mito_assembly/mito_assemblies.fasta
 seeds (chosen with constant stride from length sorted seqs) for mBed (from a total of 205 sequences)
Calculating pairwise ktuple-distances...
Ktuple-distance calculation progress done. CPU time: 2266.30u 0.15s 00:37:46.45 Elapsed: 00:18:27
ed created 4 cluster/s (with a minimum of 1 and a soft maximum of 100 sequences each)
Distance calculation within sub-clusters done. CPU time: 1553.62u 0.06s 00:25:53.67 Elapsed: 00:12:44
Guide-tree computation (mBed) done.
Progressive alignment progress done. CPU time: 2190.84u 117.80s 00:38:28.64 Elapsed: 00:35:47
Alignment written to mito.aligned.fasta


In [94]:
fasta_in = Path("mito.aligned.fasta")

# samples is assumed to already exist, e.g.:
# samples = ["sampleA", "sampleB", "sampleC"]

samples_set = set(samples)  # faster membership tests

def record_key(rec):
    return rec.id  # change to rec.description.split()[0] if needed

kept = [rec for rec in SeqIO.parse(str(fasta_in), "fasta")
        if record_key(rec) in samples_set]

# Optional: write filtered FASTA
fasta_out = Path("mito.filtered.fasta")
SeqIO.write(kept, str(fasta_out), "fasta")


178

In [95]:
%%bash

conda run -n trimal trimal -in mito.filtered.fasta -out mito.trimmed.fasta -automated1

In [96]:
in_fa = "mito.trimmed.fasta"
out_fa = "mito.raxml.fasta"

def terminal_gaps_to_missing(seq: str) -> str:
    n = len(seq)

    # leading '-'
    i = 0
    while i < n and seq[i] == "-":
        i += 1

    # trailing '-'
    j = n
    while j > i and seq[j - 1] == "-":
        j -= 1

    # seq[0:i] are leading gaps; seq[j:n] are trailing gaps; seq[i:j] is interior
    return ("?" * i) + seq[i:j] + ("?" * (n - j))

records = []
for rec in SeqIO.parse(in_fa, "fasta"):
    rec.seq = Seq(terminal_gaps_to_missing(str(rec.seq)))
    records.append(rec)

SeqIO.write(records, out_fa, "fasta")


178

In [ ]:
%%bash 

conda run -n raxml --live-stream \
    raxml-ng \
        --all \
        --msa mito.raxml.fasta \
        --msa-format FASTA \
        --model GTR \
        --tree pars{100},rand{100} \
        --threads auto \
        --workers auto \
        --bs-trees 1000 \
        --prefix mito \
        --redo


RAxML-NG v. 1.2.2 released on 11.04.2024 by The Exelixis Lab.
exandros Stamatakis. M. Kozlov and Al
ettisworth, Julia Haag, Anastasis Togkousidis.oit Morel, Sarah Lutteropp, Ben B
Latest version: https://github.com/amkozlov/raxml-ng
Questions/problems/suggestions? Please visit: https://groups.google.com/forum/#!forum/raxml

System: Intel(R) Xeon(R) Gold 6262V CPU @ 1.90GHz, 96 cores, 1007 GB RAM (allocated: 6 physical cores)

RAxML-NG was called at 12-Feb-2026 15:52:24 as follows:

-format FASTA --model GTR --tree pars{100},rand{100} --threads auto --workers auto --bs-trees 1000 --prefix mito --redo

Analysis options:
trap) mode: ML tree search + bootstrapping (Felsenstein Boots
  start tree(s): random (100) + parsimony (100)
  bootstrap replicates: parsimony (1000)
dom seed: 1770933144
  tip-inner: OFF
  pattern compression: ON
  per-rate scalers: OFF
ts: ON repea
  logLH epsilon: general: 10.000000, brlen-triplet: 1000.000000
  fast spr radius: AUTO
 subtree cutoff: 1.000000
  fast 

In [ ]:
%%bash
cp mito.raxml.support mito.raxml.support.newick
conda run -n nwed nw_ed mito.raxml.support.newick 'i & b<=50' o > mito.raxml.bs50.newick

In [ ]:
%%bash
cat ../combined/merged.csv | sed 's/,/\t/g' | cut -f1,3,4,5,17,19,22 >annotations.tsv

# Cox1 phylogeny

In [98]:
Path(f"{results_dir}/cox1_phylogeny").mkdir(parents=True, exist_ok=True)
os.chdir(f"{results_dir}/cox1_phylogeny")

In [99]:
%%bash

#get reference mito >reference_mito.fas
conda run -n entrez efetch -db nuccore -id PP905348.1 -format fasta > PP905348.1.fasta

cat ../mito_phylogeny/mito.raxml.fasta PP905348.1.fasta >cox1.fasta

In [104]:
%%bash

# I gave this an initial trim to make the alignment faster (manually)
conda run -n mito_assembly --live-stream clustalo \
    --dealign \
    -INFILE=cox1.pre-trimmed.fasta \
    -OUTFILE=cox1.aligned.fasta \
    -OUTPUT=FASTA \

FORCED DEBUG: Potential Problem: sequences (N=179) don't have same lengths but contain gaps, consider using --dealign


Using 12 threads
Read 179 sequences (type: DNA) from cox1.pre-trimmed.fasta
ed input sequences as requested.
) for mBed (from a total of 179 sequences)e from length sorted seqs
Calculating pairwise ktuple-distances...
Ktuple-distance calculation progress done. CPU time: 15.34u 0.03s 00:00:15.37 Elapsed: 00:00:07
mBed created 4 cluster/s (with a minimum of 1 and a soft maximum of 100 sequences each)
Distance calculation within sub-clusters done. CPU time: 9.74u 0.00s 00:00:09.74 Elapsed: 00:00:05
Guide-tree computation (mBed) done.
Progressive alignment progress done. CPU time: 17.51u 1.39s 00:00:18.90 Elapsed: 00:00:18
Alignment written to cox1.aligned.fasta


In [ ]:
# manually trimmed alignment to cox1 seqeunce and uploaded as cox1.raxml.fasta

In [106]:
%%bash 

conda run -n raxml --live-stream \
    raxml-ng \
        --all \
        --msa cox1.raxml.fasta \
        --msa-format FASTA \
        --model GTR \
        --tree pars{100},rand{100} \
        --bs-trees 1000 \
        --threads auto \
        --workers auto \
        --prefix cox1 \
        --redo


RAxML-NG v. 1.2.2 released on 11.04.2024 by The Exelixis Lab.
exandros Stamatakis. M. Kozlov and Al
ettisworth, Julia Haag, Anastasis Togkousidis.oit Morel, Sarah Lutteropp, Ben B
Latest version: https://github.com/amkozlov/raxml-ng
Questions/problems/suggestions? Please visit: https://groups.google.com/forum/#!forum/raxml

System: Intel(R) Xeon(R) Gold 6262V CPU @ 1.90GHz, 96 cores, 1007 GB RAM (allocated: 6 physical cores)

RAxML-NG was called at 12-Feb-2026 15:35:53 as follows:

-format FASTA --model GTR --tree pars{100},rand{100} --bs-trees 1000 --threads auto --workers auto --redo

Analysis options:
  run mode: ML tree search + bootstrapping (Felsenstein Bootstrap)
tree(s): random (100) + parsimony (100)
  bootstrap replicates: parsimony (1000)
932153om seed: 1770
  tip-inner: OFF
  pattern compression: ON
  per-rate scalers: OFF
  site repeats: ON
 epsilon: general: 10.000000, brlen-triplet: 1000.000000
  fast spr radius: AUTO
f: 1.000000ee cutof
  fast CLV updates: ON
  branch 

In [110]:
%%bash
cp cox1.raxml.support cox1.raxml.support.newick
conda run -n nwed nw_ed cox1.raxml.support.newick 'i & b<=50' o > cox1.raxml.bs50.newick

In [111]:
%%bash
cat ../combined/merged.csv | sed 's/,/\t/g' | cut -f1,3,4,5,17,19,22 >annotations.tsv